In [1]:
from QuantLib import *
import QuantLib as ql
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
class BmaScenarios:
    
    def Scen1(self,Xspreads):
        __name='Scen1'
        for n in range(1,len(Xspreads)):
        #+15bps per year for 10 years level thereafter
            Xspreads[n].setValue(-0.0015*min(n,10)) 
        return Xspreads,__name
    
    def Scen2(self,Xspreads):
        __name='Scen2'
        for n in range(1,len(Xspreads)):
        #+15bps per year for 10 years level thereafter
            Xspreads[n].setValue(+0.0015*min(n,10)) 
        return Xspreads,__name
    
    def Scen3(self,Xspreads):
        __name='Scen3'
        for n in range(1,len(Xspreads)):
        #+15bps per year for 10 years level thereafter
            Xspreads[n].setValue(+0.0025*min(n,10)) 
        return Xspreads,__name
    

In [3]:
path=r'c:\Quantlib\bma_discount2019Q1.xlsx'
todaysDate = ql.Date(30, 9, 2019)
dfbma=pd.read_excel(path,sheet_name='test')
dfbma['US']=dfbma['US']
dfbma['US'].head()
dfbma.insert(0, 'ID', range(0, len(dfbma)))
dfbma['Date']=TARGET().advance(todaysDate,0,Years)


def valuation_formula( y):
    return ql.TARGET().advance(todaysDate,y,Years)

dfbma['Date'] = dfbma.apply(lambda row: valuation_formula(row['ID']), axis=1)

df_bmaScen=dfbma[['ID','Date','US']]
df_bmaScen.head()


def extractInfoFromCurve(valDate,originScenCurve,xspreads,colNames):
    rates=[]
    
    __scenCurve = ql.SpreadedLinearZeroInterpolatedTermStructure(ql.YieldTermStructureHandle(originScenCurve),[ ql.QuoteHandle(q) for q in xspreads ],spotDates)
    for date in originScenCurve.dates():
        #nb years between valuation date and future date
        yearPassed=ql.Thirty360().yearFraction(todaysDate, date)
        #append to the list
        rates.append(__scenCurve.zeroRate(yearPassed,ql.Compounded).rate())
    #reutrns a dataframe of date & scenario curve    
    return pd.DataFrame(list(zip(list(spotCurve.dates()), rates)),columns=colNames)


In [4]:

ql.Settings.instance().evaluationDate = todaysDate
#spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 1, 2016), ql.Date(15, 1, 2017),ql.Date(15, 1, 2018)]
spotDates=[TARGET().advance(todaysDate,n,Years) for n in range(0,100)]
spotRates = dfbma['US'].tolist()[0:100]

dayCount = ql.Thirty360()
calendar = ql.UnitedStates()
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation,compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)


In [5]:
#implied forward curve
impl=ql.ImpliedTermStructure(spotCurveHandle,TARGET().advance(todaysDate,1,Years))
impl.zeroRate(1,ql.Compounded).rate()

0.02230223061704284

In [6]:


spreads = [ ql.SimpleQuote(0.0) for n in spotDates ] # null spreads to begin

ScenarioCurve = ql.SpreadedLinearZeroInterpolatedTermStructure(ql.YieldTermStructureHandle(spotCurve),[ql.QuoteHandle(q) for q in spreads],spotDates)

#scenario 1
#create the scenario spread
for n in range(1,len(spreads)):
        spreads[n].setValue(-0.0015*min(n,10))

#assign to the 
scenCurve = ql.SpreadedLinearZeroInterpolatedTermStructure(ql.YieldTermStructureHandle(spotCurve),[ ql.QuoteHandle(q) for q in spreads ],spotDates)

dated=[]
for d in spotCurve.dates():
    z=ql.Thirty360().yearFraction(todaysDate, d)
    dated.append(scenCurve.zeroRate(z,ql.Compounded).rate())

#pd.DataFrame(list(zip(list(spotCurve.dates()), dated)),columns=["Maturities","Curve"],index=['']*len(rates))
dfscen1=pd.DataFrame(list(zip(list(spotCurve.dates()), dated)),columns=["Date","Scen1"])
df_bmaScen=df_bmaScen.merge(dfscen1,on=['Date'], suffixes=('','_lag'), how='left')






In [7]:
df_bmaScen.head()
dfscen1.head()

,Date,Scen1
0,"September 30th, 2019",0.000002
1,"September 30th, 2020",0.023413
2,"September 30th, 2021",0.020559
3,"September 30th, 2022",0.018297
4,"October 2nd, 2023",0.016574


In [8]:
#scenario 2
#for n in range(1,len(spreads)):
    #+15bps per year for 10 years level thereafter
#        spreads[n].setValue(+0.0015*min(n,10))

mybma=BmaScenarios()

spreads,colname=mybma.Scen2(spreads)
df2=extractInfoFromCurve(todaysDate,spotCurve,spreads,["Date",colname])
df_bmaScen=df_bmaScen.merge(df2,on=['Date'], suffixes=('','_lag'), how='left')



In [9]:

df_bmaScen['delta']=df_bmaScen['Scen2']-df_bmaScen['US']
df_bmaScen.head(5)

,ID,Date,US,Scen1,Scen2,delta
0,0,"September 30th, 2019",0.000000,0.000002,0.000003,0.000003
1,1,"September 30th, 2020",0.024949,0.023413,0.026488,0.001539
2,2,"September 30th, 2021",0.023625,0.020559,0.026700,0.003075
3,3,"September 30th, 2022",0.022890,0.018297,0.027503,0.004613
4,4,"October 2nd, 2023",0.022692,0.016574,0.028847,0.006155


In [10]:
#implied forward curve
scenCurve = ql.SpreadedLinearZeroInterpolatedTermStructure(ql.YieldTermStructureHandle(spotCurve),[ ql.QuoteHandle(q) for q in spreads ],spotDates)
#impl=ql.ImpliedTermStructure(spotCurveHandle,TARGET().advance(todaysDate,1,Years))
#impl.zeroRate(1,ql.Compounded).rate()
ss=ql.YieldTermStructureHandle(scenCurve)

todaysDate2 = ql.Date(30, 9, 2020)
impl2=ql.ImpliedTermStructure(ss,TARGET().advance(todaysDate2,1,Years))
impl=ql.ImpliedTermStructure(spotCurveHandle,TARGET().advance(todaysDate2,1,Years))

print(impl2.zeroRate(1,ql.Compounded).rate())
print(impl.zeroRate(1,ql.Compounded).rate())

print(spotCurveHandle.forwardRate(todaysDate2,TARGET().advance(todaysDate2,2,Years),ql.Thirty360(),ql.Compounded).rate())
print(ss.forwardRate(todaysDate2,TARGET().advance(todaysDate2,2,Years),ql.Thirty360(),ql.Compounded).rate())



0.029109834661118272
0.02142038239144517
0.02186121137681285
0.028010808989128666


In [20]:
cf=ql.SimpleCashFlow(1000,ql.Date(30, 9, 2020))

cf.amount()*ss.discount(cf.date())
#0.03059001480662915-0.02288950271205059


974.1957293965547

In [74]:
spotCurveHandle.zeroRate(1,ql.Compounded).rate()
spotCurveHandle.forwardRate(ql.Date(15, 1, 2016),ql.Date(15, 1, 2017),ql.Thirty360(),ql.Compounded).rate()


0.04009803921568622

In [34]:
#loop through curve
xDates = [ql.Date(15, 1, 2015), ql.Date(15, 1, 2016), ql.Date(15, 1, 2017)]
fRates=[spotCurveHandle.forwardRate(d,TARGET().advance(d,1,Years),ql.Thirty360(),ql.Compounded).rate() for d in xDates ]
set(zip(xDates,fRates))
spotCurveHandle.discount(ql.Date(15, 1, 2018))

0.8889963586709149

In [76]:
future=todaysDate+ql.Period(1,ql.Years)
implied=ql.ImpliedTermStructure(spotCurveHandle,future)
implied.zeroRate(1,ql.Continuous).rate()

0.03931497718690904

In [5]:
#Schedule generation
effective_date = ql.Date(1, 1, 2015)
termination_date = ql.Date(1, 1, 2025)
tenor = ql.Period(ql.Annual)
calendar = ql.UnitedStates()
business_convention = ql.Following
termination_business_convention = ql.Following
date_generation = ql.DateGeneration.Forward
end_of_month = True
schedule = ql.Schedule(effective_date,
                             termination_date,
                             tenor,
                             calendar,
                             business_convention,termination_business_convention,
                             date_generation,
                             end_of_month)
pd.DataFrame({'date': list(schedule)})

,date
0,"January 2nd, 2015"
1,"January 4th, 2016"
2,"January 3rd, 2017"
3,"January 2nd, 2018"
4,"January 2nd, 2019"
5,"January 2nd, 2020"
6,"January 4th, 2021"
7,"January 3rd, 2022"
8,"January 3rd, 2023"
9,"January 2nd, 2024"


In [9]:

from QuantLib import *
helpers = [ SwapRateHelper(QuoteHandle(SimpleQuote(rate/100.0)),
Period(*tenor), TARGET(),
Annual, Unadjusted,
Thirty360(),
Euribor6M())
for tenor, rate in [((2,Years), 0.201),
((3,Years), 0.258),
((5,Years), 0.464),
((10,Years), 1.151),
((15,Years), 1.588)] ]

In [10]:
curve1 = PiecewiseFlatForward(0, TARGET(), helpers, Actual360())

In [13]:
dates, rates = zip(*curve1.nodes())


(0.001977769487929328,
 0.001977769487929328,
 0.0036675383554942956,
 0.007629248495439154,
 0.0184154249894188,
 0.025313247068010393)

In [14]:
curve2 = ForwardCurve(dates, rates, Actual360())